In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
import pickle



from sklearn.preprocessing import OneHotEncoder



team_names = {
    'Manchester United': 'ManchesterUtd',
    'Norwich City': 'NorwichCity',
    'Leicester City': 'LeicesterCity',
    'Everton': 'Everton',
    'Bournemouth': 'Bournemouth',
    'Chelsea': 'Chelsea',
    'Newcastle United': 'NewcastleUtd',
    'Arsenal': 'Arsenal',
    'Stoke City': 'StokeCity',
    'West Bromwich Albion': 'WestBrom',
    'Aston Villa': 'AstonVilla',
    'Southampton': 'Southampton',
    'Sunderland': 'Sunderland',
    'Watford': 'Watford',
    'West Ham United': 'WestHam',
    'Swansea City': 'SwanseaCity',
    'Tottenham Hotspur': 'Tottenham',
    'Crystal Palace': 'CrystalPalace',
    'Manchester City': 'ManchesterCity',
    'Liverpool': 'Liverpool',
    'Hull City': 'HullCity',
    'Burnley': 'Burnley',
    'Middlesbrough': 'Middlesbrough',
    'Brighton & Hove Albion': 'Brighton',
    'Huddersfield Town': 'Huddersfield',
    'Fulham': 'Fulham',
    'Wolverhampton Wanderers': 'Wolves',
    'Cardiff City': 'CardiffCity',
    'Sheffield United': 'SheffieldUtd',
    'Leeds United': 'LeedsUnited',
    'Brentford': 'Brentford',
    'Nottingham Forest': "Nott'hamForest",
    'Luton Town': 'LutonTown'
}


teams_data = pd.read_csv('teams.csv')
players_data = pd.read_csv('players.csv')
matches_data = pd.read_csv('matches.csv')

matches_data['HomeTeamName'] = matches_data['HomeTeamName'].replace(team_names)
matches_data['AwayTeamName'] = matches_data['AwayTeamName'].replace(team_names)

matches_data['HomeTeamAvgSeasonRank'] = pd.NaT
matches_data['HomeTeamTotalGames'] = pd.NaT
matches_data['HomeTeamTotalWins'] = pd.NaT
matches_data['HomeTeamTotalTies'] = pd.NaT
matches_data['HomeTeamTotalLosses'] = pd.NaT
matches_data['HomeTeamTotalGoalScored'] = pd.NaT
matches_data['HomeTeamTotalGoalAgainst'] = pd.NaT
matches_data['HomeTeamTotalGoalDiff'] = pd.NaT
matches_data['HomeTeamTotalPoints'] = pd.NaT


matches_data['AwayTeamAvgSeasonRank'] = pd.NaT
matches_data['AwayTeamTotalGames'] = pd.NaT
matches_data['AwayTeamTotalWins'] = pd.NaT
matches_data['AwayTeamTotalTies'] = pd.NaT
matches_data['AwayTeamTotalLosses'] = pd.NaT
matches_data['AwayTeamTotalGoalScored'] = pd.NaT
matches_data['AwayTeamTotalGoalAgainst'] = pd.NaT
matches_data['AwayTeamTotalGoalDiff'] = pd.NaT
matches_data['AwayTeamTotalPoints'] = pd.NaT

def homeTeamStats(x):
    # homeTeam = teams_data[(teams_data['Team'] == x['HomeTeamName']) & (teams_data['Season'] == ('2022-2023' if x['Season'] == '2023-2024' else x['Season']))]
    # awayTeam = teams_data[(teams_data['Team'] == x['HomeTeamName']) & (teams_data['Season'] == ('2022-2023' if x['Season'] == '2023-2024' else x['Season']))]
    if x['HomeTeamName'] == 'LutonTown': 
        x['HomeTeamName'] = 'Burnley'
    if x['AwayTeamName'] == 'LutonTown': 
        x['AwayTeamName'] = 'Burnley'
    homeTeam = teams_data[teams_data['Team'] == x['HomeTeamName']]
    awayTeam = teams_data[teams_data['Team'] == x['AwayTeamName']]
    x['HomeTeamAvgSeasonRank'] = homeTeam['SeasonRank'].sum() / homeTeam['SeasonRank'].shape[0]
    x['HomeTeamTotalGames'] = homeTeam['Games'].sum()
    x['HomeTeamTotalWins'] = homeTeam['Wins'].sum()
    x['HomeTeamTotalTies'] = homeTeam['Ties'].sum()
    x['HomeTeamTotalLosses'] = homeTeam['Losses'].sum()
    x['HomeTeamTotalGoalScored'] = homeTeam['GoalScored'].sum()
    x['HomeTeamTotalGoalAgainst'] = homeTeam['GoalAgainst'].sum()
    x['HomeTeamTotalGoalDiff'] = homeTeam['GoalScored'].sum() - homeTeam['GoalAgainst'].sum()
    x['HomeTeamTotalPoints'] = homeTeam['Points'].sum()


    x['AwayTeamAvgSeasonRank'] = awayTeam['SeasonRank'].sum() / awayTeam['SeasonRank'].shape[0]
    x['AwayTeamTotalGames'] = awayTeam['Games'].sum()
    x['AwayTeamTotalWins'] = awayTeam['Wins'].sum()
    x['AwayTeamTotalTies'] = awayTeam['Ties'].sum()
    x['AwayTeamTotalLosses'] = awayTeam['Losses'].sum()
    x['AwayTeamTotalGoalScored'] = awayTeam['GoalScored'].sum()
    x['AwayTeamTotalGoalAgainst'] = awayTeam['GoalAgainst'].sum()
    x['AwayTeamTotalGoalDiff'] = awayTeam['GoalScored'].sum() - awayTeam['GoalAgainst'].sum()
    x['AwayTeamTotalPoints'] = awayTeam['Points'].sum()
    return x


matches_data = matches_data.apply(homeTeamStats, axis=1)

encoder = OneHotEncoder(sparse=False, drop='first')

dummy_coded = encoder.fit_transform(matches_data[['Season']])

dummy_coded_df = pd.DataFrame(dummy_coded, columns=encoder.get_feature_names_out(['Season']))
matches_data = pd.concat([matches_data, dummy_coded_df], axis=1)
matches_data = matches_data.drop(columns=['Season'])

dummy_coded = encoder.fit_transform(matches_data[['Referee']])
dummy_coded_df = pd.DataFrame(dummy_coded, columns=encoder.get_feature_names_out(['Referee']))


matches_data = pd.concat([matches_data, dummy_coded_df], axis=1)

matches_data = matches_data.drop(columns=['Referee'])

X = matches_data.drop(columns=['MatchID', 'HomeTeamScore', 'AwayTeamScore', 'MatchWeek', 'HomeTeamName', 'AwayTeamName', 'Score', 'MatchDate', 'MatchTime', 'MatchResult', 'HomeTeamCaptainID', 'AwayTeamCaptainID'])
y = matches_data[['HomeTeamScore', 'AwayTeamScore']] 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


model_home = LinearRegression()
model_home.fit(X_train, y_train['HomeTeamScore'])

model_away = LinearRegression()
model_away.fit(X_train, y_train['AwayTeamScore'])

y_pred_home = model_home.predict(X_test)
y_pred_away = model_away.predict(X_test)

mse_home = mean_squared_error(y_test['HomeTeamScore'], y_pred_home)
mse_away = mean_squared_error(y_test['AwayTeamScore'], y_pred_away)

print(f'Mean Squared Error Home: {mse_home}')
print(f'Mean Squared Error Away: {mse_away}')

with open('modelHomeTeamLinear.pickle', 'wb') as f:
    pickle.dump(model_home, f)
with open('modelAwayTeamLinear.pickle', 'wb') as f:
    pickle.dump(model_away, f)
col = list(X_train.columns)
with open('colLinearModel.pickle', 'wb') as f:
    pickle.dump(col, f) 



Mean Squared Error Home: 0.18721398339590162
Mean Squared Error Away: 0.14548669706454537


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
